### Collecting Fashion House Tweets

In [19]:
import time, pandas as pd, numpy as np
import csv
import glob
import os

In [20]:
#Reading in application keys 

with open('../data/twitter_keys.txt') as api:
    keys = api.read().split('\n') 

#Aplication keys

import twitter, re, datetime, pandas as pd, numpy as np

twitter_keys = {
    'consumer_key':        keys[0],
    'consumer_secret':     keys[1],
    'access_token_key':    keys[2],
    'access_token_secret': keys[3]
}

api = twitter.Api(
    consumer_key         =   twitter_keys['consumer_key'],
    consumer_secret      =   twitter_keys['consumer_secret'],
    access_token_key     =   twitter_keys['access_token_key'],
    access_token_secret  =   twitter_keys['access_token_secret'],
    tweet_mode           =   'extended'
)


In [21]:
#Twitter API Class

class TweetMiner():
    
    result_limit = 20
    api          = None
    tweets       = []
    
    def __init__(self, keys_dict, api, result_limit = 20):
        
        self.keys_dict     = keys_dict
        self.api           = api
        self.results_limit = result_limit
        
    def mine_user_tweets(self, user='gucci', mine_retweets = False, max_pages = None, last_tweet_id = False):
        
        #time out feedback
        print("Mining tweets for:", user)
        
        tweets        = []
        
        page           = 1
        
        while page <= max_pages:
            
            #time out feedback
            print("Mining page:", page)
            
            if last_tweet_id:
                statuses = self.api.GetUserTimeline(
                    screen_name = user,
                    count       = self.result_limit,
                    max_id      = last_tweet_id - 1
                )
            else:
                statuses = self.api.GetUserTimeline(
                    screen_name = user,
                    count       = self.result_limit
                )

            for item in statuses:
                
                last_tweet_id = item.id
                
                tweets.append({
                    'tweet_id': item.id,
                    'handle':   item.user.name,
                    'text':     item.full_text,
                    'created':  item.created_at
                })
                
            page += 1
        
        return tweets
            
        
miner = TweetMiner(twitter_keys, api, result_limit=50)

### Fashion Labels

In [22]:
wikipedia_labels = ['akris_official', 'McQueen', 'alfreddunhill', 'AlphaIndustries', 'andres_sarda', 'Anne_Fontaine',\
                   'aquascutum', 'giorgioarmani', 'emporioarmani', 'ArmaniExchange', 'AspinalofLondon', 'AspreyLondon',\
                   'AleDellAcqua', 'Dior', 'dkny', 'dolcegabbana', '_BadgleyMischka', 'BadgleyMischka', 'BALENCIAGA',\
                   'Bally', 'Balmain', 'BensonAndClegg', 'berluti', 'Beulahlondon', 'ManoloBlahnik', 'HUGOBOSS',\
                    'BottegaVeneta', 'ThomBrowneNY', 'Bulgariofficial', 'Burberry', 'NigelCabourn', 'CalvinKlein',\
                   'camillaandmarc', 'Cartier', 'Dsquared2', 'CERRUTI1881', 'CHANEL', 'Childrensalon', 'chloefashion',\
                   'matthewbridal', 'chromeheartsusa', 'ClementsRibeiro', 'Coach', 'COMMEGARCONS', 'Common Projects',\
                   'corneliajames', 'Corneliani_com', 'CuteCircuit', 'cutlerandgross', 'DegeandSkinner', 'NUMEROVENTUNO',\
                   'dkny', 'dolcegabbana', 'ermannoscervino', 'ESCADA', 'EtroOfficial', 'Faconnable', 'Fendi',\
                    'TOMFORD', 'franksorbier', 'freywilleglobal', 'STUDIO_FULTON', 'DVF', 'JPGaultier', 'GievesLondon',\
                   'givenchy', 'Goyard', 'gucci', 'HackettLondon', 'KevanHallDesign', 'halston', 'Hardy Amies ', 'HELMUTLANG',\
                   'Hermes_Paris', 'HouseofHerrera', 'TommyHilfiger', 'SherriHill', 'GeorgesHobeika', 'marcjacobs', 'jimmychoo',\
                   'johnvarvatos', 'ChristopherKane', 'Karen_Kane', 'katespadeny', 'MaryKatrantzou', ' Khaadi', 'LACOSTE',\
                   'BNYhandbags', 'KarlLagerfeld', 'DerekLamNYC', 'LANVINofficial', 'LardiniOfficial', 'guylaroche', 'Larusmiani',\
                   'JudithLeiber', 'M_Lhuillier', '31philliplim', 'DanLiuTatsuaki', 'Longchamp', 'LouboutinWorld',\
                   'Luxottica', 'BrunoMagli', 'Margiela', 'MarchesaFashion', 'Marimekkoglobal', 'marniofficial',\
                   'maxmara', 'StellaMcCartney', 'MCMtweets', 'MichaelKors', 'Missoni', 'MIUMIUofficial', 'Moschino',\
                   'mouawadjewelry', 'MoynatParis', 'Mugler', 'MulberryEngland', 'charlottes_web', 'RICKOWENSONLINE',\
                   'PauleKa', 'philipp_plein', 'Prada', 'PringleScotland', 'EmilioPucci', 'PacoRabanne', 'Raja Fashions',\
                   'ralphandrusso', 'RalphLauren', 'OscardelaRenta', 'NinaRicci', 'Roberto_Cavalli', 'OfficialRodarte',\
                   'narcisostudio', 'RolandMouret', 'sergiorossi', 'MrRalphRucci', 'soniarykiel', 'sehkelly', 'Ferragamo',\
                   'sass_and_bide', 'SatyaPaulIndia', 'ITSJEREMYSCOTT', 'shanghaitang', 'JohnSmedley', 'PaulSmithDesign',\
                   'Smythson', 'JILLSTUART_JP', 'annasui', 'VivienneTam', 'THEROW', 'tibi', 'TiffanyAndCo', 'toryburch',\
                   'TrussardiNews', 'tumblerandtipsy', 'Turnbull_Asser', 'MaisonValentino', 'DriesVanNoten', 'ALEXVAUTHIER',\
                   'Versace', 'Viktor_and_Rolf', 'LouisVuitton', 'AlexanderWangNY', 'VeraWangGang', 'FollowWestwood',\
                   'MWWorld', 'JasonWu', 'thewunderkindco', 'YSL', 'giuseppezanotti', 'Zegna']


fashion_labels = ['gucci', 'McQueen', 'StellaMcCartney', 'Fendi', 'MaisonValentino', 'Roberto_Cavalli',\
                  'LouboutinWorld', 'EtroOfficial', 'Prada', 'dolcegabbana', 'Dior', 'LouisVuitton', 'CHANEL'\
                 'Missoni', 'OscardelaRenta', 'VeraWangGang', 'HerveLeger', 'MaisonValentino', 'Balmain',\
                 'Versace', 'giorgioarmani', 'DerekLamNYC', 'AlbertaFerretti', 'YSL', 'COMMEGARCONS', 'BALENCIAGA',\
                 'Burberry', 'TOMFORD', 'ALEXVAUTHIER', 'MIUMIUofficial', 'CalvinKlein', 'givenchy', 'TommyHilfiger',\
                 'jimmychoo', 'OffWht', 'chloefashion','dkny', 'marcjacobs', 'MichaelKors', 'Goyard', 'HUGOBOSS',\
                 'kenzo', 'HELMUTLANG', 'acnestudios', 'Bulgariofficial', 'akris_official']

prime_labels = ['gucci', 'McQueen', 'StellaMcCartney', 'Roberto_Cavalli', 'acnestudios', 'jimmychoo', 'Roberto_Cavalli',\
               'TOMFORD', 'Prada', 'akris_official']

print('wikipedia_labels', len(wikipedia_labels))
print('fashion_labels:', len(fashion_labels))
print('prime_labels:', len(prime_labels))

wikipedia_labels 171
fashion_labels: 45
prime_labels: 10


### Collecting and storing data

In [ ]:
# Instantiating class, passing list of twitter handles, saving tweets to csv by handle 

def scrape_label(label):
    try:
        tweet_data = miner.mine_user_tweets(user=label, max_pages=200, last_tweet_id=True)
        df = pd.DataFrame(tweet_data)
        df.to_csv('../tweet_csv/%s.csv' % label)
    except:
        print('Timed out on:', label)
        with open('../tweet_csv/0_logfile.txt', 'a+') as f:
            f.write(f'Timed out on {label} at {time.ctime()}\n')
        time.sleep(905)
        scrape_label(label)
        
for label in fashion_labels:
    scrape_label(label)         

In [8]:
# Instantiating class, passing list of twitter handles, saving tweets to csv by handle 

def scrape_label(label):
    try:
        tweet_data = miner.mine_user_tweets(user=label, max_pages=200, last_tweet_id=True)
        df = pd.DataFrame(tweet_data)
        df.to_csv('../test/%s.csv' % label)
    except:
        print('Timed out on:', label)
        with open('../test/0_logfile.txt', 'a+') as f:
            f.write(f'Timed out on {label} at {time.ctime()}\n')
        time.sleep(945)
        scrape_label(label)
        
for label in wikipedia_labels:
    scrape_label(label)      

Mining tweets for: akris_official
Mining page: 1
Mining page: 2
Mining page: 3
Mining page: 4
Mining page: 5
Mining page: 6
Mining page: 7
Mining page: 8
Mining page: 9
Mining page: 10
Mining page: 11
Mining page: 12
Mining page: 13
Mining page: 14
Mining page: 15
Mining page: 16
Mining page: 17
Mining page: 18
Mining page: 19
Mining page: 20
Mining page: 21
Mining page: 22
Mining page: 23
Mining page: 24
Mining page: 25
Mining page: 26
Mining page: 27
Mining page: 28
Mining page: 29
Mining page: 30
Mining page: 31
Mining page: 32
Mining page: 33
Mining page: 34
Mining page: 35
Mining page: 36
Mining page: 37
Mining page: 38
Mining page: 39
Mining page: 40
Mining page: 41
Mining page: 42
Mining page: 43
Mining page: 44
Mining page: 45
Mining page: 46
Mining page: 47
Mining page: 48
Mining page: 49
Mining page: 50
Mining page: 51
Mining page: 52
Mining page: 53
Mining page: 54
Mining page: 55
Mining page: 56
Mining page: 57
Mining page: 58
Mining page: 59
Mining page: 60
Mining page: 61

Mining page: 97
Mining page: 98
Mining page: 99
Mining page: 100
Mining page: 101
Mining page: 102
Mining page: 103
Mining page: 104
Mining page: 105
Mining page: 106
Mining page: 107
Mining page: 108
Mining page: 109
Mining page: 110
Mining page: 111
Mining page: 112
Mining page: 113
Mining page: 114
Mining page: 115
Mining page: 116
Mining page: 117
Mining page: 118
Mining page: 119
Mining page: 120
Mining page: 121
Mining page: 122
Mining page: 123
Mining page: 124
Mining page: 125
Mining page: 126
Mining page: 127
Mining page: 128
Mining page: 129
Mining page: 130
Mining page: 131
Mining page: 132
Mining page: 133
Mining page: 134
Mining page: 135
Mining page: 136
Mining page: 137
Mining page: 138
Mining page: 139
Mining page: 140
Mining page: 141
Mining page: 142
Mining page: 143
Mining page: 144
Mining page: 145
Mining page: 146
Mining page: 147
Mining page: 148
Mining page: 149
Mining page: 150
Mining page: 151
Mining page: 152
Mining page: 153
Mining page: 154
Mining page: 155


Mining page: 90
Mining page: 91
Mining page: 92
Mining page: 93
Mining page: 94
Mining page: 95
Mining page: 96
Mining page: 97
Mining page: 98
Mining page: 99
Mining page: 100
Mining page: 101
Mining page: 102
Mining page: 103
Mining page: 104
Mining page: 105
Mining page: 106
Mining page: 107
Mining page: 108
Mining page: 109
Mining page: 110
Mining page: 111
Mining page: 112
Mining page: 113
Mining page: 114
Mining page: 115
Mining page: 116
Mining page: 117
Mining page: 118
Mining page: 119
Mining page: 120
Mining page: 121
Mining page: 122
Mining page: 123
Mining page: 124
Mining page: 125
Mining page: 126
Mining page: 127
Mining page: 128
Mining page: 129
Mining page: 130
Mining page: 131
Mining page: 132
Mining page: 133
Mining page: 134
Mining page: 135
Mining page: 136
Mining page: 137
Mining page: 138
Mining page: 139
Mining page: 140
Mining page: 141
Mining page: 142
Mining page: 143
Mining page: 144
Mining page: 145
Mining page: 146
Mining page: 147
Mining page: 148
Mining 

Mining page: 182
Mining page: 183
Mining page: 184
Mining page: 185
Mining page: 186
Mining page: 187
Mining page: 188
Mining page: 189
Mining page: 190
Mining page: 191
Mining page: 192
Mining page: 193
Mining page: 194
Mining page: 195
Mining page: 196
Mining page: 197
Mining page: 198
Mining page: 199
Mining page: 200
Mining tweets for: giorgioarmani
Mining page: 1
Mining page: 2
Mining page: 3
Mining page: 4
Mining page: 5
Mining page: 6
Mining page: 7
Mining page: 8
Mining page: 9
Mining page: 10
Mining page: 11
Mining page: 12
Mining page: 13
Mining page: 14
Mining page: 15
Mining page: 16
Mining page: 17
Mining page: 18
Mining page: 19
Mining page: 20
Mining page: 21
Mining page: 22
Mining page: 23
Mining page: 24
Mining page: 25
Mining page: 26
Mining page: 27
Mining page: 28
Mining page: 29
Mining page: 30
Mining page: 31
Mining page: 32
Mining page: 33
Mining page: 34
Mining page: 35
Mining page: 36
Mining page: 37
Mining page: 38
Mining page: 39
Mining page: 40
Mining page: 

Mining page: 175
Mining page: 176
Mining page: 177
Mining page: 178
Mining page: 179
Mining page: 180
Mining page: 181
Mining page: 182
Mining page: 183
Mining page: 184
Mining page: 185
Mining page: 186
Mining page: 187
Mining page: 188
Mining page: 189
Mining page: 190
Mining page: 191
Mining page: 192
Mining page: 193
Mining page: 194
Mining page: 195
Mining page: 196
Mining page: 197
Mining page: 198
Mining page: 199
Mining page: 200
Mining tweets for: ArmaniExchange
Mining page: 1
Mining page: 2
Mining page: 3
Mining page: 4
Mining page: 5
Mining page: 6
Mining page: 7
Mining page: 8
Mining page: 9
Mining page: 10
Mining page: 11
Mining page: 12
Mining page: 13
Mining page: 14
Mining page: 15
Mining page: 16
Mining page: 17
Mining page: 18
Mining page: 19
Mining page: 20
Mining page: 21
Mining page: 22
Mining page: 23
Mining page: 24
Mining page: 25
Mining page: 26
Mining page: 27
Mining page: 28
Mining page: 29
Mining page: 30
Mining page: 31
Mining page: 32
Mining page: 33
Minin

...

Mining page: 15
Mining page: 16
Mining page: 17
Mining page: 18
Mining page: 19
Mining page: 20
Mining page: 21
Mining page: 22
Mining page: 23
Mining page: 24
Mining page: 25
Mining page: 26
Mining page: 27
Mining page: 28
Mining page: 29
Mining page: 30
Mining page: 31
Mining page: 32
Mining page: 33
Mining page: 34
Mining page: 35
Mining page: 36
Mining page: 37
Mining page: 38
Mining page: 39
Mining page: 40
Mining page: 41
Mining page: 42
Mining page: 43
Mining page: 44
Mining page: 45
Mining page: 46
Mining page: 47
Mining page: 48
Mining page: 49
Mining page: 50
Mining page: 51
Mining page: 52
Mining page: 53
Mining page: 54
Mining page: 55
Mining page: 56
Mining page: 57
Mining page: 58
Mining page: 59
Mining page: 60
Mining page: 61
Mining page: 62
Mining page: 63
Mining page: 64
Mining page: 65
Mining page: 66
Mining page: 67
Mining page: 68
Mining page: 69
Mining page: 70
Mining page: 71
Mining page: 72
Mining page: 73
Mining page: 74
Mining page: 75
Mining page: 76
Mining p

Mining page: 7
Mining page: 8
Mining page: 9
Mining page: 10
Mining page: 11
Mining page: 12
Mining page: 13
Mining page: 14
Mining page: 15
Mining page: 16
Mining page: 17
Mining page: 18
Mining page: 19
Mining page: 20
Mining page: 21
Mining page: 22
Mining page: 23
Mining page: 24
Mining page: 25
Mining page: 26
Mining page: 27
Mining page: 28
Mining page: 29
Mining page: 30
Mining page: 31
Mining page: 32
Mining page: 33
Mining page: 34
Mining page: 35
Mining page: 36
Mining page: 37
Mining page: 38
Mining page: 39
Mining page: 40
Mining page: 41
Mining page: 42
Mining page: 43
Mining page: 44
Mining page: 45
Mining page: 46
Mining page: 47
Mining page: 48
Mining page: 49
Mining page: 50
Mining page: 51
Mining page: 52
Mining page: 53
Mining page: 54
Mining page: 55
Mining page: 56
Mining page: 57
Mining page: 58
Mining page: 59
Mining page: 60
Mining page: 61
Mining page: 62
Mining page: 63
Mining page: 64
Mining page: 65
Mining page: 66
Mining page: 67
Mining page: 68
Mining page

Mining page: 2
Mining page: 3
Mining page: 4
Mining page: 5
Mining page: 6
Mining page: 7
Mining page: 8
Mining page: 9
Mining page: 10
Mining page: 11
Mining page: 12
Mining page: 13
Mining page: 14
Mining page: 15
Mining page: 16
Mining page: 17
Mining page: 18
Mining page: 19
Mining page: 20
Mining page: 21
Mining page: 22
Mining page: 23
Mining page: 24
Mining page: 25
Mining page: 26
Mining page: 27
Mining page: 28
Mining page: 29
Mining page: 30
Mining page: 31
Mining page: 32
Mining page: 33
Mining page: 34
Mining page: 35
Mining page: 36
Mining page: 37
Mining page: 38
Mining page: 39
Mining page: 40
Mining page: 41
Mining page: 42
Mining page: 43
Mining page: 44
Mining page: 45
Mining page: 46
Mining page: 47
Mining page: 48
Mining page: 49
Mining page: 50
Mining page: 51
Mining page: 52
Mining page: 53
Mining page: 54
Mining page: 55
Mining page: 56
Mining page: 57
Mining page: 58
Mining page: 59
Mining page: 60
Mining page: 61
Mining page: 62
Mining page: 63
Mining page: 64


Mining page: 100
Mining page: 101
Mining page: 102
Mining page: 103
Mining page: 104
Mining page: 105
Mining page: 106
Mining page: 107
Mining page: 108
Mining page: 109
Mining page: 110
Mining page: 111
Mining page: 112
Mining page: 113
Mining page: 114
Mining page: 115
Mining page: 116
Mining page: 117
Mining page: 118
Mining page: 119
Mining page: 120
Mining page: 121
Mining page: 122
Mining page: 123
Mining page: 124
Mining page: 125
Mining page: 126
Mining page: 127
Mining page: 128
Mining page: 129
Mining page: 130
Mining page: 131
Mining page: 132
Mining page: 133
Mining page: 134
Mining page: 135
Mining page: 136
Mining page: 137
Mining page: 138
Mining page: 139
Mining page: 140
Mining page: 141
Mining page: 142
Mining page: 143
Mining page: 144
Mining page: 145
Mining page: 146
Mining page: 147
Mining page: 148
Mining page: 149
Mining page: 150
Mining page: 151
Mining page: 152
Mining page: 153
Mining page: 154
Mining page: 155
Mining page: 156
Mining page: 157
Mining page: 1

Mining page: 93
Mining page: 94
Mining page: 95
Mining page: 96
Mining page: 97
Mining page: 98
Mining page: 99
Mining page: 100
Mining page: 101
Mining page: 102
Mining page: 103
Mining page: 104
Mining page: 105
Mining page: 106
Mining page: 107
Mining page: 108
Mining page: 109
Mining page: 110
Mining page: 111
Mining page: 112
Mining page: 113
Mining page: 114
Mining page: 115
Mining page: 116
Mining page: 117
Mining page: 118
Mining page: 119
Mining page: 120
Mining page: 121
Mining page: 122
Mining page: 123
Mining page: 124
Mining page: 125
Mining page: 126
Mining page: 127
Mining page: 128
Mining page: 129
Mining page: 130
Mining page: 131
Mining page: 132
Mining page: 133
Mining page: 134
Mining page: 135
Mining page: 136
Mining page: 137
Mining page: 138
Mining page: 139
Mining page: 140
Mining page: 141
Mining page: 142
Mining page: 143
Mining page: 144
Mining page: 145
Mining page: 146
Mining page: 147
Mining page: 148
Mining page: 149
Mining page: 150
Mining page: 151
Mini

In [40]:
#reading in fashion house csv and creating master datafram

acne_df      = pd.read_csv('../tweet_csv/acnestudios.csv',index_col=[0])
akris_df     = pd.read_csv('../tweet_csv/akris_official.csv',index_col=[0])
alberta_df   = pd.read_csv('../tweet_csv/AlbertaFerretti.csv',index_col=[0])
alexvauth_df = pd.read_csv('../tweet_csv/ALEXVAUTHIER.csv',index_col=[0])
balenci_df   = pd.read_csv('../tweet_csv/BALENCIAGA.csv',index_col=[0])

balmain_df   = pd.read_csv('../tweet_csv/Balmain.csv',index_col=[0])
bulgari_df   = pd.read_csv('../tweet_csv/Bulgariofficial.csv',index_col=[0])
burbery_df   = pd.read_csv('../tweet_csv/Burberry.csv',index_col=[0])
calvin_df    = pd.read_csv('../tweet_csv/CalvinKlein.csv',index_col=[0])
chanel_df    = pd.read_csv('../tweet_csv/CHANEL.csv',index_col=[0])

chloe_df     = pd.read_csv('../tweet_csv/chloefashion.csv',index_col=[0])
commes_df    = pd.read_csv('../tweet_csv/COMMEGARCONS.csv',index_col=[0])
dereklam_df  = pd.read_csv('../tweet_csv/DerekLamNYC.csv',index_col=[0])
dior_df      = pd.read_csv('../tweet_csv/Dior.csv',index_col=[0])
dkny_df      = pd.read_csv('../tweet_csv/dkny.csv',index_col=[0])

dolce_df     = pd.read_csv('../tweet_csv/dolcegabbana.csv',index_col=[0])
etro_df      = pd.read_csv('../tweet_csv/EtroOfficial.csv',index_col=[0])
fendi_df     = pd.read_csv('../tweet_csv/Fendi.csv',index_col=[0])
armani_df    = pd.read_csv('../tweet_csv/giorgioarmani.csv',index_col=[0])
givenchy_df  = pd.read_csv('../tweet_csv/givenchy.csv',index_col=[0])

goyard_df    = pd.read_csv('../tweet_csv/Goyard.csv',index_col=[0])
gucci_df     = pd.read_csv('../tweet_csv/gucci.csv',index_col=[0])
helmut_df    = pd.read_csv('../tweet_csv/HELMUTLANG.csv',index_col=[0])
herve_df     = pd.read_csv('../tweet_csv/HerveLeger.csv',index_col=[0])
hugoboss_df  = pd.read_csv('../tweet_csv/HUGOBOSS.csv',index_col=[0])

jimmychoo_df = pd.read_csv('../tweet_csv/jimmychoo.csv',index_col=[0])
lagerfeld_df = pd.read_csv('../tweet_csv/KarlLagerfeld.csv',index_col=[0])
kenzo_df     = pd.read_csv('../tweet_csv/kenzo.csv',index_col=[0])
lanvin_df    = pd.read_csv('../tweet_csv/LANVINofficial.csv',index_col=[0])
Louboutin_df = pd.read_csv('../tweet_csv/LouboutinWorld.csv',index_col=[0])

louisv_df    = pd.read_csv('../tweet_csv/LouisVuitton.csv',index_col=[0])
valentino_df = pd.read_csv('../tweet_csv/MaisonValentino.csv',index_col=[0])
marcjacob_df = pd.read_csv('../tweet_csv/marcjacobs.csv',index_col=[0])
mcqueen_df   = pd.read_csv('../tweet_csv/McQueen.csv',index_col=[0])
kors_df      = pd.read_csv('../tweet_csv/MichaelKors.csv',index_col=[0])

missoni_df   = pd.read_csv('../tweet_csv/Missoni.csv',index_col=[0])
miumiu_df    = pd.read_csv('../tweet_csv/MIUMIUofficial.csv',index_col=[0])
offwht_df    = pd.read_csv('../tweet_csv/OffWht.csv',index_col=[0])
oscar_df     = pd.read_csv('../tweet_csv/OscardelaRenta.csv',index_col=[0])
prada_df     = pd.read_csv('../tweet_csv/Prada.csv',index_col=[0])

cavalli_df   = pd.read_csv('../tweet_csv/Roberto_Cavalli.csv',index_col=[0])
stella_df    = pd.read_csv('../tweet_csv/StellaMcCartney.csv',index_col=[0])
tomford_df   = pd.read_csv('../tweet_csv/TOMFORD.csv',index_col=[0])
hilfiger_df  = pd.read_csv('../tweet_csv/TommyHilfiger.csv',index_col=[0])
verawang_df  = pd.read_csv('../tweet_csv/VeraWangGang.csv',index_col=[0])

versache_df  = pd.read_csv('../tweet_csv/Versace.csv',index_col=[0])
ysl_df       = pd.read_csv('../tweet_csv/YSL.csv',index_col=[0])


df_list = [acne_df, akris_df, alberta_df, alexvauth_df, balenci_df, balmain_df, bulgari_df, burbery_df, calvin_df,\
           chanel_df, chloe_df, commes_df, dereklam_df, dior_df, dkny_df, dolce_df, etro_df, fendi_df,\
           armani_df, givenchy_df, goyard_df, gucci_df, helmut_df, herve_df, hugoboss_df, jimmychoo_df,\
           lagerfeld_df, kenzo_df, lanvin_df, Louboutin_df, louisv_df, valentino_df, marcjacob_df, mcqueen_df,\
           kors_df, missoni_df, miumiu_df, offwht_df, oscar_df, prada_df, cavalli_df, stella_df, tomford_df,\
           hilfiger_df, verawang_df, versache_df, ysl_df]

fashion_df = pd.concat(df_list, ignore_index=True)
fashion_df.shape

(113881, 4)

In [27]:
#merged all csv's in tweet_csv directory using termianl comand "cat *.csv >merged.csv"

fashion_df = pd.read_csv('../tweet_csv/__merged.csv', index_col=[0])
fashion_df.shape

(113927, 4)

In [41]:
fashion_df.groupby(fashion_df.handle).size().sort_values(ascending=False)

handle
Roberto Cavalli        3247
Dolce & Gabbana        3245
Vera Wang              3244
Valentino              3240
Stella McCartney       3238
Missoni                3238
Oscar de la Renta      3234
Michael Kors           3234
VERSACE                3232
HUGO BOSS              3231
gucci                  3231
Marc Jacobs            3226
CALVIN KLEIN           3225
Fendi                  3224
AlbertaFerretti        3221
COMME des GARÇONS      3219
ETRO                   3217
Burberry               3216
Giorgio Armani         3212
Alexander McQueen      3209
Louis Vuitton          3209
Tommy Hilfiger         3205
Christian Louboutin    3204
Balmain                3204
Off-White™             3175
Derek Lam              3014
Bulgari                2853
Chloé                  2813
LANVIN                 2704
Dior                   2567
PRADA                  2005
KARL LAGERFELD         1978
CHANEL                 1819
Herve Leger            1409
DKNY                   1278
MIU MIU      

#### Saving Merged Dataframe 

- each account's tweets are allocated to an individual csv that is then loaded into a master dataframe containing all tweets.

In [42]:
fashion_df.to_pickle('../data/fashion_df.pkl')